In [ ]:
import stacrs
import pystac
from odc.stac import load
from arraylake import Client as ArrayLakeClient
from dask.distributed import Client as DaskClient


In [ ]:
url = "https://data.source.coop/ausantarctic/ghrsst-mur-v2/ghrsst-mur-v2.parquet"

dicts = await stacrs.read(url)
items = [pystac.Item.from_dict(d) for d in dicts["features"]]
less_items = [i for i in items if i.datetime.year == 2025]


In [ ]:
less_items = [i for i in items if i.datetime.year == 2025]

print(f"Loaded {len(items)} items, filtered to {len(less_items)} items")

In [ ]:
data = load(
    items,
    chunks={"longitude": 4000, "latitude": 4000, "time": 1},
    measurements=["analysed_sst"],
    fail_on_error=False
)

data

In [ ]:
# Filter the xarray to data just for 2025
data_2025 = data.sel(time=slice("2025-01-01", "2025-12-31"))
data_2025

In [ ]:
# Instantiate the Arraylake client
client = ArrayLakeClient()

# Checkout the repo
repo = client.get_repo("AustralianAntarcticDivision/ghrsst-testing")
session = repo.writable_session("main")

In [ ]:
# # Set up dask
# dask_client = DaskClient()
# dask_client

In [ ]:
# Create some data with Xarray
data_2025.to_zarr(session.store, group="analysed_sst", zarr_format=3, mode="a", append_dim="time")

# Make your first commit
session.commit('Initial Commit')

In [ ]:
import xarray as xr

client = ArrayLakeClient()
repo = client.get_repo("AustralianAntarcticDivision/ghrsst-testing")
session = repo.readonly_session("main")

ds = xr.open_zarr(session.store, group="analysed_sst", consolidated=True)
ds